#property strict

input double PriceStepMult = 1.0;
input color  PocColor      = clrOrange;
input int    PocWidth      = 2;

// ─────────────────────────────────────
double step;
double pLow, pHigh;
double volMap[];
int    mapSize = 0;

bool   initialized = false;
string pocName = "DEV_POC_LINE";

// ─────────────────────────────────────
int OnInit()
{
   step = _Point * PriceStepMult;

   ObjectDelete(0, pocName);
   ObjectCreate(0, pocName, OBJ_POLYLINE, 0);
   ObjectSetInteger(0, pocName, OBJPROP_COLOR, PocColor);
   ObjectSetInteger(0, pocName, OBJPROP_WIDTH, PocWidth);

   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
void OnDeinit(const int reason)
{
   ObjectDelete(0, pocName);
}

// ─────────────────────────────────────
void OnTick()
{
   double price = SymbolInfoDouble(_Symbol, SYMBOL_BID);
   datetime t   = TimeCurrent();

   // --- initialize
   if(!initialized)
   {
      pLow = pHigh = price;
      mapSize = 1;
      ArrayResize(volMap, 1);
      volMap[0] = 1.0;

      initialized = true;
      AppendPoint(t, price);
      return;
   }

   // --- expand map if needed
   if(price < pLow)
   {
      int add = int((pLow - price) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pLow -= add * step;
   }
   else if(price > pHigh)
   {
      int add = int((price - pHigh) / step) + 1;
      int old = mapSize;
      mapSize += add;

      ArrayResize(volMap, mapSize);
      for(int i = old; i < mapSize; i++)
         volMap[i] = 0.0;

      pHigh += add * step;
   }

   int idx = int((price - pLow) / step);
   if(idx >= 0 && idx < mapSize)
      volMap[idx] += 1.0;

   // --- compute POC
   double maxVol = 0.0;
   int pocIdx = 0;

   for(int i = 0; i < mapSize; i++)
